## Preprocessing data
Preprocessing dilakukan setelah melakukan pelabelan, karena pelabelan dilakukan manual. Bertujuan untuk memudahkan pemberian kategori sentimen saat pelabelan data

In [2]:
# Load hasil labeling
import pandas as pd
data = pd.read_csv("../Klasifikasi/DatasetLabeled.csv")
data.head()

,full_text,teks_bersih,label
0,@CakKhum Satu lagi mentri laknatullah alaih......,CakKhum Satu lagi mentri laknatullah alaih Uru...,negative
1,@hanyarbabini @anunemouse @jimans_ yakin perta...,hanyarbabini anunemouse jimans_ yakin pertamin...,neutral
2,Udah di oplos Ada etanol Harga naik Dan pertam...,Udah oplos etanol Harga naik pertamina tetap r...,negative
3,@susno2g @Danantara_ri @riekediahp @pertamina ...,susno2g Danantara_ri riekediahp pertamina Keme...,NaN
4,Tertibkan dong @pertamina jangan rusuh etanol aja,Tertibkan dong pertamina jangan rusuh etanol,negative


In [14]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory


#1. Casefolding data
def caseFolding(data, text_field, new_text_field_name):
    # Mengubah huruf kapital menjadi kecil
    data[new_text_field_name] = data[text_field].str.lower()
    # Menghapus karakter seperti @, #
    data[new_text_field_name] = data[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    # Menghapus angka
    data[new_text_field_name] = data[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    return data

data_clean = caseFolding(data, 'teks_bersih', 'clean_text')
data_clean.head(10)


,full_text,teks_bersih,label,clean_text
0,@CakKhum Satu lagi mentri laknatullah alaih......,CakKhum Satu lagi mentri laknatullah alaih Uru...,negative,cakkhum satu lagi mentri laknatullah alaih uru...
1,@hanyarbabini @anunemouse @jimans_ yakin perta...,hanyarbabini anunemouse jimans_ yakin pertamin...,neutral,hanyarbabini anunemouse jimans yakin pertamina...
2,Udah di oplos Ada etanol Harga naik Dan pertam...,Udah oplos etanol Harga naik pertamina tetap r...,negative,udah oplos etanol harga naik pertamina tetap r...
3,@susno2g @Danantara_ri @riekediahp @pertamina ...,susno2g Danantara_ri riekediahp pertamina Keme...,NaN,susnog danantarari riekediahp pertamina kement...
4,Tertibkan dong @pertamina jangan rusuh etanol aja,Tertibkan dong pertamina jangan rusuh etanol,negative,tertibkan dong pertamina jangan rusuh etanol
5,Rieke Diah Pitaloka mengkritik impor etanol ol...,Rieke Diah Pitaloka mengkritik impor etanol ol...,neutral,rieke diah pitaloka mengkritik impor etanol ol...
6,@Jul3Prikitiw huh. gua. feromonnya. Pertamina....,Jul3Prikitiw feromonnya Pertamina Etanol,NaN,julprikitiw feromonnya pertamina etanol
7,Pertamina Bidik Peluang dari Etanol: 70 Persen...,Pertamina Bidik Peluang dari Etanol Persen Leb...,NaN,pertamina bidik peluang dari etanol persen leb...
8,@susno2g @Kemenperin_RI @kementan @Kementerian...,susno2g Kemenperin_RI kementan KementerianESDM...,NaN,susnog kemenperinri kementan kementerianesdm p...
9,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,negative,rieke diah pitaloka mempertanyakan kesiapan pe...


In [17]:
# # 2. Normalisasi kata tidak baku
# def replace_taboo_words(text, kamus_tidak_baku):
#     if isinstance(text, str):
#         words = text.split()
#         replaced_words = []
#         kalimat_baku = []
#         kata_diganti = []
#         kata_tidak_baku_hash = []

#         for word in words:
#             if word in kamus_tidak_baku:
#                 baku_word = kamus_tidak_baku[word]
#                 if isinstance(baku_word, str) and all(char.isalpha() or char.isspace() for char in baku_word):
#                     replaced_words.append(baku_word)
#                     kalimat_baku.append(baku_word)
#                     kata_diganti.append(word)
#                     kata_tidak_baku_hash.append(hash(word))
#                 else:
#                     replaced_words.append(word)
#             else:
#                 replaced_words.append(word)

#         replaced_text = ' '.join(replaced_words)
#         return replaced_text, kalimat_baku, kata_diganti, kata_tidak_baku_hash
#     else:
#         return "", [], [], []

# # Load kamus data
# kamus_data = pd.read_csv("../kamus/kamuskatabaku.csv")
# kamus_tidak_baku = dict(zip(kamus_data["tidak_baku"], kamus_data["kata_baku"]))
# results = data["clean_text"].apply(lambda x: replace_taboo_words(x, kamus_tidak_baku))
# data["normalisasi_text"], data["kata_baku"], data["kata_tidak_baku"], data["kata_tidak_baku_Hash"] = zip(*results)
# data[['full_text', 'teks_bersih', 'clean_text', 'normalisasi_text']].head(10)


# 2. Normalisasi kata tidak baku
def replace_taboo_words(text, kamus_tidak_baku):
    if isinstance(text, str):
        words = text.split()
        replaced_words = []
        kalimat_baku = []
        kata_diganti = []
        kata_tidak_baku_hash = []

        for word in words:
            if word in kamus_tidak_baku:
                baku_word = kamus_tidak_baku[word]
                if isinstance(baku_word, str) and all(char.isalpha() or char.isspace() for char in baku_word):
                    replaced_words.append(baku_word)
                    kalimat_baku.append(baku_word)
                    kata_diganti.append(word)
                    kata_tidak_baku_hash.append(hash(word))
                else:
                    replaced_words.append(word)
            else:
                replaced_words.append(word)

        replaced_text = ' '.join(replaced_words)
        return replaced_text, kalimat_baku, kata_diganti, kata_tidak_baku_hash
    else:
        return "", [], [], []


# ==== Perbaikan DI SINI ====
kamus_data = pd.read_csv("../kamus/kamuskatabaku.csv", sep=';')
kamus_data["tidak_baku"] = kamus_data["tidak_baku"].astype(str).str.strip()
kamus_data["kata_baku"]  = kamus_data["kata_baku"].astype(str).str.strip()
kamus_tidak_baku = dict(zip(kamus_data["tidak_baku"], kamus_data["kata_baku"]))

# Terapkan normalisasi
results = data["clean_text"].apply(lambda x: replace_taboo_words(x, kamus_tidak_baku))

data["normalisasi_text"], data["kata_baku"], data["kata_tidak_baku"], data["kata_tidak_baku_Hash"] = zip(*results)

data[['full_text', 'teks_bersih', 'clean_text', 'normalisasi_text']].head(10)


,full_text,teks_bersih,clean_text,normalisasi_text
0,@CakKhum Satu lagi mentri laknatullah alaih......,CakKhum Satu lagi mentri laknatullah alaih Uru...,cakkhum satu lagi mentri laknatullah alaih uru...,cakkhum satu lagi mentri laknatullah alaih uru...
1,@hanyarbabini @anunemouse @jimans_ yakin perta...,hanyarbabini anunemouse jimans_ yakin pertamin...,hanyarbabini anunemouse jimans yakin pertamina...,hanyarbabini anunemouse jimans yakin pertamina...
2,Udah di oplos Ada etanol Harga naik Dan pertam...,Udah oplos etanol Harga naik pertamina tetap r...,udah oplos etanol harga naik pertamina tetap r...,sudah oplos etanol harga naik pertamina tetap ...
3,@susno2g @Danantara_ri @riekediahp @pertamina ...,susno2g Danantara_ri riekediahp pertamina Keme...,susnog danantarari riekediahp pertamina kement...,susnog danantarari riekediahp pertamina kement...
4,Tertibkan dong @pertamina jangan rusuh etanol aja,Tertibkan dong pertamina jangan rusuh etanol,tertibkan dong pertamina jangan rusuh etanol,tertibkan dong pertamina jangan rusuh etanol
5,Rieke Diah Pitaloka mengkritik impor etanol ol...,Rieke Diah Pitaloka mengkritik impor etanol ol...,rieke diah pitaloka mengkritik impor etanol ol...,rieke diah pitaloka mengkritik impor etanol ol...
6,@Jul3Prikitiw huh. gua. feromonnya. Pertamina....,Jul3Prikitiw feromonnya Pertamina Etanol,julprikitiw feromonnya pertamina etanol,julprikitiw feromonnya pertamina etanol
7,Pertamina Bidik Peluang dari Etanol: 70 Persen...,Pertamina Bidik Peluang dari Etanol Persen Leb...,pertamina bidik peluang dari etanol persen leb...,pertamina bidik peluang dari etanol persen leb...
8,@susno2g @Kemenperin_RI @kementan @Kementerian...,susno2g Kemenperin_RI kementan KementerianESDM...,susnog kemenperinri kementan kementerianesdm p...,susnog kemenperinri kementan kementerianesdm p...
9,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,rieke diah pitaloka mempertanyakan kesiapan pe...,rieke diah pitaloka mempertanyakan kesiapan pe...


In [18]:
# 3. tokenizing (memisahkan teks menjadi potongan potongan)

def tokenize(text):
    tokens = text.split()
    return tokens

data["tokenize"] = data["normalisasi_text"].apply(tokenize)
data.head(10)


,full_text,teks_bersih,label,clean_text,normalisasi_text,kata_baku,kata_tidak_baku,kata_tidak_baku_Hash,tokenize
0,@CakKhum Satu lagi mentri laknatullah alaih......,CakKhum Satu lagi mentri laknatullah alaih Uru...,negative,cakkhum satu lagi mentri laknatullah alaih uru...,cakkhum satu lagi mentri laknatullah alaih uru...,[],[],[],"[cakkhum, satu, lagi, mentri, laknatullah, ala..."
1,@hanyarbabini @anunemouse @jimans_ yakin perta...,hanyarbabini anunemouse jimans_ yakin pertamin...,neutral,hanyarbabini anunemouse jimans yakin pertamina...,hanyarbabini anunemouse jimans yakin pertamina...,[],[],[],"[hanyarbabini, anunemouse, jimans, yakin, pert..."
2,Udah di oplos Ada etanol Harga naik Dan pertam...,Udah oplos etanol Harga naik pertamina tetap r...,negative,udah oplos etanol harga naik pertamina tetap r...,sudah oplos etanol harga naik pertamina tetap ...,[sudah],[udah],[-7837224959496172808],"[sudah, oplos, etanol, harga, naik, pertamina,..."
3,@susno2g @Danantara_ri @riekediahp @pertamina ...,susno2g Danantara_ri riekediahp pertamina Keme...,NaN,susnog danantarari riekediahp pertamina kement...,susnog danantarari riekediahp pertamina kement...,[],[],[],"[susnog, danantarari, riekediahp, pertamina, k..."
4,Tertibkan dong @pertamina jangan rusuh etanol aja,Tertibkan dong pertamina jangan rusuh etanol,negative,tertibkan dong pertamina jangan rusuh etanol,tertibkan dong pertamina jangan rusuh etanol,[],[],[],"[tertibkan, dong, pertamina, jangan, rusuh, et..."
5,Rieke Diah Pitaloka mengkritik impor etanol ol...,Rieke Diah Pitaloka mengkritik impor etanol ol...,neutral,rieke diah pitaloka mengkritik impor etanol ol...,rieke diah pitaloka mengkritik impor etanol ol...,[],[],[],"[rieke, diah, pitaloka, mengkritik, impor, eta..."
6,@Jul3Prikitiw huh. gua. feromonnya. Pertamina....,Jul3Prikitiw feromonnya Pertamina Etanol,NaN,julprikitiw feromonnya pertamina etanol,julprikitiw feromonnya pertamina etanol,[],[],[],"[julprikitiw, feromonnya, pertamina, etanol]"
7,Pertamina Bidik Peluang dari Etanol: 70 Persen...,Pertamina Bidik Peluang dari Etanol Persen Leb...,NaN,pertamina bidik peluang dari etanol persen leb...,pertamina bidik peluang dari etanol persen leb...,[],[],[],"[pertamina, bidik, peluang, dari, etanol, pers..."
8,@susno2g @Kemenperin_RI @kementan @Kementerian...,susno2g Kemenperin_RI kementan KementerianESDM...,NaN,susnog kemenperinri kementan kementerianesdm p...,susnog kemenperinri kementan kementerianesdm p...,[],[],[],"[susnog, kemenperinri, kementan, kementerianes..."
9,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,negative,rieke diah pitaloka mempertanyakan kesiapan pe...,rieke diah pitaloka mempertanyakan kesiapan pe...,[],[],[],"[rieke, diah, pitaloka, mempertanyakan, kesiap..."


In [19]:
#  4. Stopword removal (menghapus kata umum atau sambung contoh yang, di, dan)

# Unduh stopwords bahasa Indonesia
nltk.download('stopwords')

# Mendapatkan stopwords bahasa Indonesia
stop_words = stopwords.words('indonesian')
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

data['stopword_removal'] = data['tokenize'].apply(lambda x: remove_stopwords(x))
data.head(10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,full_text,teks_bersih,label,clean_text,normalisasi_text,kata_baku,kata_tidak_baku,kata_tidak_baku_Hash,tokenize,stopword_removal
0,@CakKhum Satu lagi mentri laknatullah alaih......,CakKhum Satu lagi mentri laknatullah alaih Uru...,negative,cakkhum satu lagi mentri laknatullah alaih uru...,cakkhum satu lagi mentri laknatullah alaih uru...,[],[],[],"[cakkhum, satu, lagi, mentri, laknatullah, ala...","[cakkhum, mentri, laknatullah, alaih, urusan, ..."
1,@hanyarbabini @anunemouse @jimans_ yakin perta...,hanyarbabini anunemouse jimans_ yakin pertamin...,neutral,hanyarbabini anunemouse jimans yakin pertamina...,hanyarbabini anunemouse jimans yakin pertamina...,[],[],[],"[hanyarbabini, anunemouse, jimans, yakin, pert...","[hanyarbabini, anunemouse, jimans, pertamina, ..."
2,Udah di oplos Ada etanol Harga naik Dan pertam...,Udah oplos etanol Harga naik pertamina tetap r...,negative,udah oplos etanol harga naik pertamina tetap r...,sudah oplos etanol harga naik pertamina tetap ...,[sudah],[udah],[-7837224959496172808],"[sudah, oplos, etanol, harga, naik, pertamina,...","[oplos, etanol, harga, pertamina, rugi, angkas..."
3,@susno2g @Danantara_ri @riekediahp @pertamina ...,susno2g Danantara_ri riekediahp pertamina Keme...,NaN,susnog danantarari riekediahp pertamina kement...,susnog danantarari riekediahp pertamina kement...,[],[],[],"[susnog, danantarari, riekediahp, pertamina, k...","[susnog, danantarari, riekediahp, pertamina, k..."
4,Tertibkan dong @pertamina jangan rusuh etanol aja,Tertibkan dong pertamina jangan rusuh etanol,negative,tertibkan dong pertamina jangan rusuh etanol,tertibkan dong pertamina jangan rusuh etanol,[],[],[],"[tertibkan, dong, pertamina, jangan, rusuh, et...","[tertibkan, pertamina, rusuh, etanol]"
5,Rieke Diah Pitaloka mengkritik impor etanol ol...,Rieke Diah Pitaloka mengkritik impor etanol ol...,neutral,rieke diah pitaloka mengkritik impor etanol ol...,rieke diah pitaloka mengkritik impor etanol ol...,[],[],[],"[rieke, diah, pitaloka, mengkritik, impor, eta...","[rieke, diah, pitaloka, mengkritik, impor, eta..."
6,@Jul3Prikitiw huh. gua. feromonnya. Pertamina....,Jul3Prikitiw feromonnya Pertamina Etanol,NaN,julprikitiw feromonnya pertamina etanol,julprikitiw feromonnya pertamina etanol,[],[],[],"[julprikitiw, feromonnya, pertamina, etanol]","[julprikitiw, feromonnya, pertamina, etanol]"
7,Pertamina Bidik Peluang dari Etanol: 70 Persen...,Pertamina Bidik Peluang dari Etanol Persen Leb...,NaN,pertamina bidik peluang dari etanol persen leb...,pertamina bidik peluang dari etanol persen leb...,[],[],[],"[pertamina, bidik, peluang, dari, etanol, pers...","[pertamina, bidik, peluang, etanol, persen, mu..."
8,@susno2g @Kemenperin_RI @kementan @Kementerian...,susno2g Kemenperin_RI kementan KementerianESDM...,NaN,susnog kemenperinri kementan kementerianesdm p...,susnog kemenperinri kementan kementerianesdm p...,[],[],[],"[susnog, kemenperinri, kementan, kementerianes...","[susnog, kemenperinri, kementan, kementerianes..."
9,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,negative,rieke diah pitaloka mempertanyakan kesiapan pe...,rieke diah pitaloka mempertanyakan kesiapan pe...,[],[],[],"[rieke, diah, pitaloka, mempertanyakan, kesiap...","[rieke, diah, pitaloka, kesiapan, pertamina, m..."


In [20]:
 # Step 5: Stemming

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(text):
    return [stemmer.stem(word) for word in text]

data['steming_data'] = data['stopword_removal'].apply(lambda x:' '.join(stem_text(x)))
data.head(10)

,full_text,teks_bersih,label,clean_text,normalisasi_text,kata_baku,kata_tidak_baku,kata_tidak_baku_Hash,tokenize,stopword_removal,steming_data
0,@CakKhum Satu lagi mentri laknatullah alaih......,CakKhum Satu lagi mentri laknatullah alaih Uru...,negative,cakkhum satu lagi mentri laknatullah alaih uru...,cakkhum satu lagi mentri laknatullah alaih uru...,[],[],[],"[cakkhum, satu, lagi, mentri, laknatullah, ala...","[cakkhum, mentri, laknatullah, alaih, urusan, ...",cakkhum tri laknatullah alaih urus etanol koru...
1,@hanyarbabini @anunemouse @jimans_ yakin perta...,hanyarbabini anunemouse jimans_ yakin pertamin...,neutral,hanyarbabini anunemouse jimans yakin pertamina...,hanyarbabini anunemouse jimans yakin pertamina...,[],[],[],"[hanyarbabini, anunemouse, jimans, yakin, pert...","[hanyarbabini, anunemouse, jimans, pertamina, ...",hanyarbabini anunemouse jimans pertamina perse...
2,Udah di oplos Ada etanol Harga naik Dan pertam...,Udah oplos etanol Harga naik pertamina tetap r...,negative,udah oplos etanol harga naik pertamina tetap r...,sudah oplos etanol harga naik pertamina tetap ...,[sudah],[udah],[-7837224959496172808],"[sudah, oplos, etanol, harga, naik, pertamina,...","[oplos, etanol, harga, pertamina, rugi, angkas...",oplos etanol harga pertamina rugi angkasa bobrok
3,@susno2g @Danantara_ri @riekediahp @pertamina ...,susno2g Danantara_ri riekediahp pertamina Keme...,NaN,susnog danantarari riekediahp pertamina kement...,susnog danantarari riekediahp pertamina kement...,[],[],[],"[susnog, danantarari, riekediahp, pertamina, k...","[susnog, danantarari, riekediahp, pertamina, k...",susnog danantarari riekediahp pertamina kement...
4,Tertibkan dong @pertamina jangan rusuh etanol aja,Tertibkan dong pertamina jangan rusuh etanol,negative,tertibkan dong pertamina jangan rusuh etanol,tertibkan dong pertamina jangan rusuh etanol,[],[],[],"[tertibkan, dong, pertamina, jangan, rusuh, et...","[tertibkan, pertamina, rusuh, etanol]",tertib pertamina rusuh etanol
5,Rieke Diah Pitaloka mengkritik impor etanol ol...,Rieke Diah Pitaloka mengkritik impor etanol ol...,neutral,rieke diah pitaloka mengkritik impor etanol ol...,rieke diah pitaloka mengkritik impor etanol ol...,[],[],[],"[rieke, diah, pitaloka, mengkritik, impor, eta...","[rieke, diah, pitaloka, mengkritik, impor, eta...",rieke diah pitaloka kritik impor etanol pertam...
6,@Jul3Prikitiw huh. gua. feromonnya. Pertamina....,Jul3Prikitiw feromonnya Pertamina Etanol,NaN,julprikitiw feromonnya pertamina etanol,julprikitiw feromonnya pertamina etanol,[],[],[],"[julprikitiw, feromonnya, pertamina, etanol]","[julprikitiw, feromonnya, pertamina, etanol]",julprikitiw feromon pertamina etanol
7,Pertamina Bidik Peluang dari Etanol: 70 Persen...,Pertamina Bidik Peluang dari Etanol Persen Leb...,NaN,pertamina bidik peluang dari etanol persen leb...,pertamina bidik peluang dari etanol persen leb...,[],[],[],"[pertamina, bidik, peluang, dari, etanol, pers...","[pertamina, bidik, peluang, etanol, persen, mu...",pertamina bidik peluang etanol persen murah be...
8,@susno2g @Kemenperin_RI @kementan @Kementerian...,susno2g Kemenperin_RI kementan KementerianESDM...,NaN,susnog kemenperinri kementan kementerianesdm p...,susnog kemenperinri kementan kementerianesdm p...,[],[],[],"[susnog, kemenperinri, kementan, kementerianes...","[susnog, kemenperinri, kementan, kementerianes...",susnog kemenperinri tan kementerianesdm pertam...
9,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,negative,rieke diah pitaloka mempertanyakan kesiapan pe...,rieke diah pitaloka mempertanyakan kesiapan pe...,[],[],[],"[rieke, diah, pitaloka, mempertanyakan, kesiap...","[rieke, diah, pitaloka, kesiapan, pertamina, m...",rieke diah pitaloka kesiap pertamina serap eta...


In [22]:
import pandas as pd

data = pd.DataFrame(data[["full_text","clean_text","normalisasi_text","tokenize","stopword_removal","steming_data","label"]])
data.head(10)

,full_text,clean_text,normalisasi_text,tokenize,stopword_removal,steming_data,label
0,@CakKhum Satu lagi mentri laknatullah alaih......,cakkhum satu lagi mentri laknatullah alaih uru...,cakkhum satu lagi mentri laknatullah alaih uru...,"[cakkhum, satu, lagi, mentri, laknatullah, ala...","[cakkhum, mentri, laknatullah, alaih, urusan, ...",cakkhum tri laknatullah alaih urus etanol koru...,negative
1,@hanyarbabini @anunemouse @jimans_ yakin perta...,hanyarbabini anunemouse jimans yakin pertamina...,hanyarbabini anunemouse jimans yakin pertamina...,"[hanyarbabini, anunemouse, jimans, yakin, pert...","[hanyarbabini, anunemouse, jimans, pertamina, ...",hanyarbabini anunemouse jimans pertamina perse...,neutral
2,Udah di oplos Ada etanol Harga naik Dan pertam...,udah oplos etanol harga naik pertamina tetap r...,sudah oplos etanol harga naik pertamina tetap ...,"[sudah, oplos, etanol, harga, naik, pertamina,...","[oplos, etanol, harga, pertamina, rugi, angkas...",oplos etanol harga pertamina rugi angkasa bobrok,negative
3,@susno2g @Danantara_ri @riekediahp @pertamina ...,susnog danantarari riekediahp pertamina kement...,susnog danantarari riekediahp pertamina kement...,"[susnog, danantarari, riekediahp, pertamina, k...","[susnog, danantarari, riekediahp, pertamina, k...",susnog danantarari riekediahp pertamina kement...,NaN
4,Tertibkan dong @pertamina jangan rusuh etanol aja,tertibkan dong pertamina jangan rusuh etanol,tertibkan dong pertamina jangan rusuh etanol,"[tertibkan, dong, pertamina, jangan, rusuh, et...","[tertibkan, pertamina, rusuh, etanol]",tertib pertamina rusuh etanol,negative
5,Rieke Diah Pitaloka mengkritik impor etanol ol...,rieke diah pitaloka mengkritik impor etanol ol...,rieke diah pitaloka mengkritik impor etanol ol...,"[rieke, diah, pitaloka, mengkritik, impor, eta...","[rieke, diah, pitaloka, mengkritik, impor, eta...",rieke diah pitaloka kritik impor etanol pertam...,neutral
6,@Jul3Prikitiw huh. gua. feromonnya. Pertamina....,julprikitiw feromonnya pertamina etanol,julprikitiw feromonnya pertamina etanol,"[julprikitiw, feromonnya, pertamina, etanol]","[julprikitiw, feromonnya, pertamina, etanol]",julprikitiw feromon pertamina etanol,NaN
7,Pertamina Bidik Peluang dari Etanol: 70 Persen...,pertamina bidik peluang dari etanol persen leb...,pertamina bidik peluang dari etanol persen leb...,"[pertamina, bidik, peluang, dari, etanol, pers...","[pertamina, bidik, peluang, etanol, persen, mu...",pertamina bidik peluang etanol persen murah be...,NaN
8,@susno2g @Kemenperin_RI @kementan @Kementerian...,susnog kemenperinri kementan kementerianesdm p...,susnog kemenperinri kementan kementerianesdm p...,"[susnog, kemenperinri, kementan, kementerianes...","[susnog, kemenperinri, kementan, kementerianes...",susnog kemenperinri tan kementerianesdm pertam...,NaN
9,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,rieke diah pitaloka mempertanyakan kesiapan pe...,rieke diah pitaloka mempertanyakan kesiapan pe...,"[rieke, diah, pitaloka, mempertanyakan, kesiap...","[rieke, diah, pitaloka, kesiapan, pertamina, m...",rieke diah pitaloka kesiap pertamina serap eta...,negative


In [30]:
import os

folder_tujuan = 'Klasifikasi'
os.makedirs(folder_tujuan, exist_ok=True)

path_output = os.path.join(folder_tujuan, 'DatasetLabeledClean.csv')
data.to_csv(path_output, index=False, encoding='utf-8')

print("File disimpan di:", path_output)


File disimpan di: Klasifikasi\DatasetLabeledClean.csv


## Menyiapkan Data label

In [31]:
import pandas as pd

# Load hasil labeling
df = pd.read_csv("../Klasifikasi/DatasetLabeledClean.csv")

# Lihat jumlah per label
print(df["label"].value_counts(dropna=False))

# Filter berdasarkan label
df_negatif = df[df["label"] == "negative"]
df_positif = df[df["label"] == "positive"]
df_netral  = df[df["label"] == "neutral"]

# Ambil baris yang labelnya kosong / NaN / whitespace
df_unlabeled = df[df["label"].isna() | (df["label"].str.strip() == "")]

# Simpan ke file terpisah
df_negatif.to_csv("negative.csv", index=False)
df_positif.to_csv("positive.csv", index=False)
df_netral.to_csv("neutral.csv", index=False)
df_unlabeled.to_csv("unlabeled.csv", index=False)


label
NaN         892
negative    100
positive    100
neutral      50
Name: count, dtype: int64


In [33]:
import pandas as pd
import glob
import os

# Tentukan folder
path_folder = ''

# Pola file CSV yang ingin digabung
daftar_file_csv = [
    os.path.join(path_folder, "negative.csv"),
    os.path.join(path_folder, "neutral.csv"),
    os.path.join(path_folder, "positive.csv")
]

# Filter hanya file yang benar-benar ada
daftar_file_csv = [f for f in daftar_file_csv if os.path.exists(f)]

if not daftar_file_csv:
    print(f"Tidak ada file CSV yang ditemukan di folder: {os.path.abspath(path_folder)}")
else:
    print(f"Menemukan {len(daftar_file_csv)} file CSV untuk digabungkan:")
    print(daftar_file_csv)

    # 1. Gabungkan semua file
    list_df = []
    for filename in daftar_file_csv:
        print(f"Membaca file: {filename}...")
        df = pd.read_csv(filename)
        list_df.append(df)

    data = pd.concat(list_df, ignore_index=True)
    print(f"\nTotal baris data gabungan: {len(data)}")

    # 2. Simpan hasil gabungan
    path_output = os.path.join(path_folder, 'Dataset250Labeled.csv')
    data.to_csv(path_output, index=False, encoding='utf-8')

    print(f"\nFile berhasil disimpan di: {path_output}")

    # 3. Tampilkan info data
    print("\nInfo dari DataFrame yang disimpan:")
    data.info()


Menemukan 3 file CSV untuk digabungkan:
['negative.csv', 'neutral.csv', 'positive.csv']
Membaca file: negative.csv...
Membaca file: neutral.csv...
Membaca file: positive.csv...

Total baris data gabungan: 250

File berhasil disimpan di: Dataset250Labeled.csv

Info dari DataFrame yang disimpan:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   full_text         250 non-null    object
 1   clean_text        250 non-null    object
 2   normalisasi_text  250 non-null    object
 3   tokenize          250 non-null    object
 4   stopword_removal  250 non-null    object
 5   steming_data      250 non-null    object
 6   label             250 non-null    object
dtypes: object(7)
memory usage: 13.8+ KB


In [36]:
import pandas as pd
import os

# --- KONFIGURASI ---
nama_file_input = "../Klasifikasi/unlabeled.csv" 
nama_file_output = "../Klasifikasi/Dataset750Unlabeld.csv"

try:
    # 1. MEMBACA FILE
    print("Sedang membaca data...")
    if os.path.exists(nama_file_input):
        df = pd.read_csv(nama_file_input)
        
        total_data = len(df)
        print(f"Total data asli ditemukan: {total_data} baris.")

        # 2. PROSES PENGAMBILAN DATA (SAMPLING)
        if total_data >= 750:
            # Mengambil 1000 data secara ACAK
            df_subset = df.sample(n=750, random_state=42)
            
            # --- [BAGIAN BARU] 3. MEMILIH KOLOM TERTENTU ---
            # Gunakan kurung siku dua kali [['...']] agar tetap berbentuk Tabel (DataFrame)
            # Pastikan nama kolom di CSV Anda benar-benar "steming_data" (cek spasi/typo)
            df_final = df_subset[['steming_data','label']] 

            # 4. MENYIMPAN KE FILE BARU
            # index=False agar nomor baris tidak ikut tersimpan
            df_final.to_csv(nama_file_output, index=False)
            
            print("------------------------------------------------")
            print(f"SUKSES! Berhasil menyimpan kolom 'steming_data' saja.")
            print(f"Jumlah data: {len(df_final)}")
            print(f"File disimpan di: {nama_file_output}")
            print("------------------------------------------------")
            
        else:
            print(f"PERINGATAN: Jumlah data Anda hanya {total_data}, kurang dari 1000.")

    else:
        print(f"ERROR: File tidak ditemukan di path: {nama_file_input}")

except KeyError:
    print("ERROR: Kolom 'steming_data' tidak ditemukan di dalam CSV.")
    print("Cek kembali nama kolom di file asli Anda (apakah ada spasi atau huruf besar/kecil).")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Sedang membaca data...
Total data asli ditemukan: 892 baris.
------------------------------------------------
SUKSES! Berhasil menyimpan kolom 'steming_data' saja.
Jumlah data: 750
File disimpan di: ../Klasifikasi/Dataset750Unlabeld.csv
------------------------------------------------


## Tugas eksperimen model klasifikasi

## Load Data

In [87]:
import pandas as pd
import re
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle
from gensim.models import Word2Vec
import numpy as np
from sklearn.svm import SVC

# 1. LOAD DATA (PALING ATAS
df = pd.read_csv("../Klasifikasi/Dataset250Labeled.csv")
df.head()


,full_text,clean_text,normalisasi_text,tokenize,stopword_removal,steming_data,label
0,@CakKhum Satu lagi mentri laknatullah alaih......,cakkhum satu lagi mentri laknatullah alaih uru...,cakkhum satu lagi mentri laknatullah alaih uru...,"['cakkhum', 'satu', 'lagi', 'mentri', 'laknatu...","['cakkhum', 'mentri', 'laknatullah', 'alaih', ...",cakkhum tri laknatullah alaih urus etanol koru...,negative
1,Udah di oplos Ada etanol Harga naik Dan pertam...,udah oplos etanol harga naik pertamina tetap r...,sudah oplos etanol harga naik pertamina tetap ...,"['sudah', 'oplos', 'etanol', 'harga', 'naik', ...","['oplos', 'etanol', 'harga', 'pertamina', 'rug...",oplos etanol harga pertamina rugi angkasa bobrok,negative
2,Tertibkan dong @pertamina jangan rusuh etanol aja,tertibkan dong pertamina jangan rusuh etanol,tertibkan dong pertamina jangan rusuh etanol,"['tertibkan', 'dong', 'pertamina', 'jangan', '...","['tertibkan', 'pertamina', 'rusuh', 'etanol']",tertib pertamina rusuh etanol,negative
3,Rieke Diah Pitaloka mempertanyakan kesiapan Pe...,rieke diah pitaloka mempertanyakan kesiapan pe...,rieke diah pitaloka mempertanyakan kesiapan pe...,"['rieke', 'diah', 'pitaloka', 'mempertanyakan'...","['rieke', 'diah', 'pitaloka', 'kesiapan', 'per...",rieke diah pitaloka kesiap pertamina serap eta...,negative
4,@susno2g @pertamina @Kemenperin_RI @Kementeria...,susnog pertamina kemenperinri kementerianesdm ...,susnog pertamina kemenperinri kementerianesdm ...,"['susnog', 'pertamina', 'kemenperinri', 'kemen...","['susnog', 'pertamina', 'kemenperinri', 'kemen...",susnog pertamina kemenperinri kementerianesdm ...,negative


In [88]:
df.tail()

,full_text,clean_text,normalisasi_text,tokenize,stopword_removal,steming_data,label
245,Aku kangen pertamina hiks,kangen pertamina hiks,kangen pertamina hiks,"['kangen', 'pertamina', 'hiks']","['kangen', 'pertamina', 'hiks']",kangen pertamina hiks,positive
246,@AzzamAlifAditya Ekonomi masyarakat terus meni...,azzamalifaditya ekonomi masyarakat terus menin...,azzamalifaditya ekonomi masyarakat terus menin...,"['azzamalifaditya', 'ekonomi', 'masyarakat', '...","['azzamalifaditya', 'ekonomi', 'masyarakat', '...",azzamalifaditya ekonomi masyarakat tingkat kil...,positive
247,@Indranata_Tama Good job Kilang Pertamina Inte...,indranatatama good kilang pertamina internasional,indranatatama good kilang pertamina internasional,"['indranatatama', 'good', 'kilang', 'pertamina...","['indranatatama', 'good', 'kilang', 'pertamina...",indranatatama good kilang pertamina internasional,positive
248,@Indranata_Tama Kilang Pertamina Internasional...,indranatatama kilang pertamina internasional t...,indranatatama kilang pertamina internasional t...,"['indranatatama', 'kilang', 'pertamina', 'inte...","['indranatatama', 'kilang', 'pertamina', 'inte...",indranatatama kilang pertamina internasional baik,positive
249,@dolfasola Ini baru kerja hebat Kilang Pertami...,dolfasola baru kerja hebat kilang pertamina in...,dolfasola baru kerja hebat kilang pertamina in...,"['dolfasola', 'baru', 'kerja', 'hebat', 'kilan...","['dolfasola', 'kerja', 'hebat', 'kilang', 'per...",dolfasola kerja hebat kilang pertamina interna...,positive


## 1. Klasifikasi Naive Bayes + TF-IDF

In [89]:
# TF-IDF FEATUR
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df["steming_data"])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [78]:
# Model Naive Bayes 
print("\n Model Naive Bayes + TF-IDF ")
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_pred = nb_model.predict(X_test)
print(classification_report(y_test, nb_pred))


 Model Naive Bayes + TF-IDF 
              precision    recall  f1-score   support

    negative       0.62      0.90      0.73        20
     neutral       0.00      0.00      0.00        10
    positive       0.76      0.80      0.78        20

    accuracy                           0.68        50
   macro avg       0.46      0.57      0.51        50
weighted avg       0.55      0.68      0.61        50



c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

## 2. Klasifikasi Support Vector Machine + TF-IDF

In [79]:
# TF-IDF FEATUR
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df["steming_data"])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [80]:
# Model Support Vector Machine 
print("\n Model Support Vector Machine + TF-IDF ")
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)
print(classification_report(y_test, svm_pred))


 Model Support Vector Machine + TF-IDF 
              precision    recall  f1-score   support

    negative       0.56      0.95      0.70        20
     neutral       0.00      0.00      0.00        10
    positive       0.88      0.70      0.78        20

    accuracy                           0.66        50
   macro avg       0.48      0.55      0.49        50
weighted avg       0.57      0.66      0.59        50



c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

## 3. Klasifikasi Logistic Regression menggunakan TF-IDF

In [69]:
# TF-IDF 
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df["steming_data"])
y = df["label"]

# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [81]:
# Model Logistic Regression
print("\n Model Logistic Regression + TF-IDF ")
logreg = LogisticRegression(max_iter=300)
logreg.fit(X_train, y_train)

# 4. Prediksi
pred = logreg.predict(X_test)
print(classification_report(y_test, pred))



 Model Logistic Regression + TF-IDF 
              precision    recall  f1-score   support

    negative       0.61      0.85      0.71        20
     neutral       0.00      0.00      0.00        10
    positive       0.73      0.80      0.76        20

    accuracy                           0.66        50
   macro avg       0.44      0.55      0.49        50
weighted avg       0.53      0.66      0.59        50



c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

## 4. Klasifikasi SVM menggunakan Word2Vec

In [58]:
!pip install gensim


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [82]:
# WORD2VEC FEATURE
sentences = df["steming_data"].apply(lambda x: x.split())

# Train Word2Vec
w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Konversi kalimat → vector
def sent_vector(words):
    vectors = [w2v.wv[w] for w in words if w in w2v.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(w2v.vector_size)

X = np.array([sent_vector(s) for s in sentences])
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [84]:
# Model Support Vector Machine 
print("\n Model Support Vector Machine + Word2Vec")
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)

svm_pred = svm_model.predict(X_test)
print(classification_report(y_test, svm_pred))


 Model Support Vector Machine + Word2Vec
              precision    recall  f1-score   support

    negative       0.49      0.90      0.63        20
     neutral       0.00      0.00      0.00        10
    positive       0.54      0.35      0.42        20

    accuracy                           0.50        50
   macro avg       0.34      0.42      0.35        50
weighted avg       0.41      0.50      0.42        50



c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag